# Tasks list for lesson 1.02

Today’s dataset is a sample from the training data, for all the people who have donated to "Healthcare for All".

Instead of having all the features from the training data, we have a subset of only a few features. The objective of this sample dataset is to teach students some important steps in data blending and cleaning, tasks critical for a subsequent linear regression that predicts the target variable `TARGET_D` (we'll see this in other units).


### Scenario

Data is provided to the analyst in four different Excel files named file1, file2, file3, and file4. Data is downloaded from a server at some frequent intervals and stored into different files. All the files have the same data recorder, ie. the same headers.

We will need to merge all this data together into one file before we start with any other operation on the data.

There is another file that contains other data such as date of birth, domain, and `TARGET_D`.

Use the Date of birth column to find the age.

Use `vlook_up` from the file `vlookup_table.csv`, using the column `CONTROLN` to add the column `TARGET_D` to the previous merged data

As an exercise, split the column `Domain` and use `vlook_up` from the file `vlookup_table.csv` to check which kind of neighborhood they live in (ie. urban, city, town, rural, etc.).

After merging all the data together, start with the data cleaning operations as mentioned in the lesson.

### Leemos los ficheros de entrada con la librería Pandas

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_excel("file1.xlsx", engine='openpyxl') # quizás tengáis que instalar otra libreria llamada openpyxl (conda install -c anaconda openpyxl)

In [3]:
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')] # borramos las columnas que han aparecido como Unnamed por irrelevantes

In [11]:
df1

,CONTROLN,STATE,GENDER,HV1,IC1,IC4,HVP1,IC5,POBC1,POBC2,IC2,IC3,AVGGIFT
0,44060,FL,M,AAA896,392,520.0,7,21975,6,16,430.0,466,28.000000
1,96093,IL,M,537,365,473.0,0,19387,1,89,415.0,410,5.666667
2,43333,FL,F,725,301,436.0,3,18837,11,17,340.0,361,4.111111
3,21885,NC,M,AAA1095,401,413.0,7,14014,1,74,407.0,399,27.277778
4,190108,FL,F,995,252,348.0,0,17991,5,6,280.0,316,6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,114721,OK,NaN,1040,472,656.0,0,26962,2,56,609.0,579,11.666667
1003,149152,CA,M,4507,842,962.0,95,54642,17,50,1004.0,893,20.000000
1004,959,IA,F,586,349,465.0,1,15304,1,77,413.0,404,7.300000
1005,179563,WA,male,842,420,494.0,2,12894,41,24,419.0,476,16.400000


In [5]:
df2 = pd.read_excel("file1.xlsx", engine='openpyxl')
df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]

In [6]:
df3 = pd.read_excel("file1.xlsx", engine='openpyxl')
df3 = df3.loc[:, ~df3.columns.str.contains('^Unnamed')]

In [7]:
df4 = pd.read_excel("file1.xlsx", engine='openpyxl')
df4 = df4.loc[:, ~df4.columns.str.contains('^Unnamed')]

**Unimos (concat) los dataframes verticalmente:**

In [8]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [10]:
len(df)

4028

**Observación 1: Hay campos incompletos (aparecen como NaN en el dataframe)**

Google search: pandas drop all rows with nan

In [32]:
df = df.dropna()

**Observación 2: Hay columnas que deberían ser numéricas y contienen caracteres**

Google search: pandas drop all non numeric rows

In [33]:
col_numericas = ['CONTROLN', 'HV1',  'IC1', 'IC4', 'HVP1', 'IC5', 'POBC1', 'POBC2', 'IC2', 'IC3', 'AVGGIFT']

In [39]:
for col in col_numericas:
    df = df[pd.to_numeric(df[col], errors='coerce').notnull()]

**Observación 4: La columna STATE tiene representaciones múltiples de los estados de US**

In [42]:
df['STATE'].unique()

array(['IL', 'FL', 'NC', 'TX', 'NV', 'CA', 'Cali', 'AP', 'IN', 'MT', 'WI',
       'MO', 'HI', 'UT', 'GA', 'WA', 'ID', 'CT', 'AL', 'ND', 'SC', 'IA',
       'CO', 'LA', 'OR', 'SD', 'TN', 'NM', 'AZ', 'MN', 'KY', 'OK', 'NJ',
       'NE', 'MI', 'AR', 'KS', 'California', 'MS', 'NY', 'Arizona', 'WY',
       'Tennessee'], dtype=object)

Vemos que hay filas que representan los estados ['CA','AZ', 'TN'] como [California, Arizona, Tennessee]

Abreviaremos esas ocurrencias para darle integridad al dataset:

In [43]:
df['STATE'] = df['STATE'].str.replace('California','CA')
df['STATE'] = df['STATE'].str.replace('Cali','CA')
df['STATE'] = df['STATE'].str.replace('Arizona','AZ')
df['STATE'] = df['STATE'].str.replace('Tennessee','TN')

In [45]:
df['STATE'].unique()

array(['IL', 'FL', 'NC', 'TX', 'NV', 'CA', 'AP', 'IN', 'MT', 'WI', 'MO',
       'HI', 'UT', 'GA', 'WA', 'ID', 'CT', 'AL', 'ND', 'SC', 'IA', 'CO',
       'LA', 'OR', 'SD', 'TN', 'NM', 'AZ', 'MN', 'KY', 'OK', 'NJ', 'NE',
       'MI', 'AR', 'KS', 'MS', 'NY', 'WY'], dtype=object)

### **Ejercicio: Limpiar el dataset para hacer el join vlookup_table.csv**

Nos informan que tenemos otra archivo que contiene información adicional vlookup_table.csv

In [52]:
df_lookup = pd.read_csv('vlookup_table.csv')

In [61]:
## nos cargamos los NaN
df_lookup = df_lookup.dropna() 

## Nos cargamos las columnas numéricas que contienen strings o otros datos:
col_numericas = ['TCODE', 'DOB',  'CONTROLN', 'TARGET_D']
for col in col_numericas:
    df_lookup = df_lookup[pd.to_numeric(df_lookup[col], errors='coerce').notnull()]

## Comprovamos integridad de la columna DOMAIN:
df_lookup['DOMAIN'].unique()

## Comprovamos integridad de la columna GENDER:
df_lookup['GENDER'].unique()

# Nos cargamos los GENDER que no sabemos que significan U y J:
df_lookup = df_lookup[df_lookup["GENDER"].str.contains("U|J")==False]


### Ahora queremos calcular la edad a partir de la columna Date of Bird (DOB)

In [133]:
df_lookup['DOB'].head(20) 

0     1901
2     2501
3     2208
5     6104
6     4407
8     1801
10    5401
11    3401
12    2801
14    2405
16    2701
17    6501
18    2301
20    5107
24    4304
26    4406
27    6105
28    2401
30    2401
32    4806
Name: DOB, dtype: int64

In [103]:
df_lookup['DOB_pd'] = pd.to_datetime(df_lookup['DOB'], format='%d%m%y', errors='coerce')

In [105]:
df_lookup = df_lookup[~df_lookup['DOB_pd'].isna()]

In [106]:
today = pd.to_datetime('today')

In [107]:
today

Timestamp('2022-09-20 14:19:46.583681')

In [116]:
df_lookup.loc[:, 'AGE'] = (today-df_lookup['DOB_pd']).astype('timedelta64[Y]').astype(int)

C:\Users\JosepCarreras\anaconda3\envs\moto\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [123]:
df_lookup

,TCODE,STATE,DOB,DOMAIN,GENDER,CONTROLN,TARGET_D,DOB_pd,AGE
0,1,FL,1901,C2,M,44060,100.0,2001-09-01,21
2,0,FL,2501,C2,F,43333,5.0,2001-05-02,21
3,0,NC,2208,T2,M,21885,38.0,2008-02-02,14
5,1,IL,6104,S2,M,100640,30.0,2004-01-06,18
6,0,NC,4407,R2,F,18581,12.0,2007-04-04,15
...,...,...,...,...,...,...,...,...,...
3992,0,TX,4105,U1,F,117326,10.0,2005-01-04,17
3993,1,MI,3312,T1,M,68618,40.0,2012-03-03,10
3994,1,CA,4711,C3,M,1646,10.0,2011-07-04,11
3996,1,FL,2501,T2,M,6443,20.0,2001-05-02,21


### JOIN de las tablas

In [124]:
df_lookup_join = df_lookup[['CONTROLN', 'AGE', 'TARGET_D']]

In [131]:
df_final = df.merge(df_lookup_join, left_on='CONTROLN', right_on='CONTROLN', how='inner')

In [15]:
import os

In [17]:
print(os.getcwd())

C:\Users\JosepCarreras\ironhack\u1d2\data_case_study_1
